# TP Représentation symbolique du langage naturel et inférence

In [2]:
import mynltk
import nltk
from nltk.sem import Expression, skolemize
from nltk.inference.resolution import clausify, Clause

read_expr = nltk.sem.Expression.fromstring

Loading patch
Patching nltk.sem.logic


In [3]:
import os
os.environ['PROVER9'] = 'c:/Program Files (x86)/Prover9-Mace4/bin-win32/'


## Préliminaire

Dans cette partie, vous allez utiliser un démonstrateur automatique externe qui implémente la résolution et qui est beaucoup plus performant que celui fourni nativement par NLTK. L'interface reste la même du point de vue de Python, en revanche les temps d'exécution sont réduits de beaucoup.

Rendez-vous sur la page de téléchargement de [Prover9](https://www.cs.unm.edu/~mccune/prover9/gui/v05.html) et installez la version adaptée à votre système. Notez le répertoire d'installation et le chemin de l'exécutable `prover9` (Mac/Unix) ou `prover9.exe` (Windows).

L'interface depuis Python reste la même : vous utilisez `Prover9Command` au lieu de `ResolutionProverCommand`.

## Répondre aux questions

Lorsque vous donnez un énoncé à l'agent, celui-ci doit déterminer si cet énoncé est acceptable ou non, et lorsqu'il est acceptable, si l'agent apprend quelque chose de nouveau avec cet énoncé.

Lorsque vous posez une question fermée à l'agent, celui-ci doit déterminer s'il peut répondre par oui ou par non, ou bien si la réponse lui est inconnue.

L'agent possède une base de connaissances KB. Face à un nouveau fait f, les situations possibles sont les suivantes :

- f est une conséquence de KB ($KB \vDash f$) : l'agent répond "I already know that"
- f est en contradiction avec KB ($KB \vDash \neg f$) : l'agent répond "I don't buy that"
- f est contingent de KB (KB n'a ni $f$, ni $\neg f$ pour conséquence) : l'agent ajoute f à KB et répond "I learned something"

Dans le cas d'une question f, les situations possibles sont les suivantes :

- f est une conséquence de KB ($KB \vDash f$) : l'agent répond "Yes"
- f est en contradiction avec KB ($KB \vDash \neg f$) : l'agent répond "No"
- f est contingent de KB (KB n'a ni $f$, ni $\neg f$ pour conséquence) : l'agent répond "I don't know"


## Travail

### Partie 1

A partir de ce point, votre travail va consister à assembler un système de dialogue à partir des éléments vu précédemment, de sorte à reproduire le dialogue ciblé.

In [74]:
import mynltk
import nltk
from nltk.inference.resolution import ResolutionProverCommand
from nltk.sem.logic import AndExpression, ConstantExpression, Variable
from nltk.inference.prover9 import Prover9Command

sents = [
    'Alice is a student',     # student(alice)
    'Is Alice a student ?',   # student(alice)
    'Is Bob a student ?',     # student(bob)
    'Students are people',    # student(X) -> person(X)
    'Alice is a person',      # person(alice)
    'Alice is not a person',  # -person(alice)
    'Alice is from Phoenix',    # from(phoenix, alice)
    'Is Alice from a city ?', # from(X, alice) & city(X)
    'Phoenix is a hot city',  # city(phoenix) & hot(phoenix)
    'Is Alice from a city ?', # from(X, alice) & city(X)
    'Cities are places',      # city(X) -> place(X) 
    'If it is snowing then it is cold', # snow -> cold
    'It is not cold',
    'Is it snowing ?',        # snow
    'If a person is from a hot place and it is cold then she is not happy', # person(X) & from(Y, X) & hot(Y) & cold -> -happy(X)   
    'Is it snowing ?',        # snow
    'Alice is happy',         # happy(alice)
    'Is Alice happy ?',        # happy(alice)
    'Is it snowing ?'         # snow
    ]


Pour enchaîner le dialogue, nous avons le choix entre :
- taper les phrases une par une au clavier
- fournir automatiquement les phrases du dialogue de référence.

Pour permettre d'utiliser indifféremment l'une ou l'autre,  nous définissons deux classes sur lesquelles itérer.

In [5]:
class ReadLineIterator:
    """Class to implement an iterator
    of readline()"""

    def __init__(self, stop='bye'):
        self.stop = stop

    def __iter__(self):
        return self

    def __next__(self):
        self.input = input('>>> ')
        if self.input != self.stop:
            return self.input
        else:
            raise StopIteration

class SentencesIterator:
    """Class to implement an iterator
    of list"""

    def __init__(self, sentences):
        self.sentences = sentences

    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index < len(self.sentences):
            s = self.sentences[self.index]
            self.index += 1
            print('>>> ', s)
            return s
        else:
            raise StopIteration


Vous avez un squelette de classe agent qui va réaliser le dialogue et les réponses dans la cellule suivante.

Pour que le dialogue soit complet, il faut :
- ajouter les règles manquantes à la grammaire `dialog.fcfg` pour analyser certaines formes non traitées,
- inférer la réponse correcte par résolution en suivant les règles données plus haut.

In [77]:
class Agent:

    def __init__(self, grammar='grammar/dialog.fcfg', source=iter(SentencesIterator(sents))):
        self.KB = []
        with open(grammar, 'r') as file:
            data = file.read()
        self.grammar = nltk.grammar.FeatureGrammar.fromstring(data)
        self.parser = nltk.FeatureChartParser(self.grammar, trace=0)
        self.source = source

    def prove(self, formula):
        tp = Prover9Command(formula, self.KB)
        # ATTENTION: ajuster le chemin à l'exécutable
        tp._prover.config_prover9('c:/Program Files (x86)/Prover9-Mace4/bin-win32/prover9.exe')
        return tp.prove()

    def dialog(self):
        try:
                while True:
                    sentence = next(self.source)
                    try:
                        parsed_tree = next(self.parser.parse(sentence.split()))
                        formula = parsed_tree.label()['SEM']
                        assert isinstance(formula, AndExpression)
                        question = (formula.second == ConstantExpression(Variable('question')))
                        assertion = (formula.second == ConstantExpression(Variable('assertion')))
                        formula = formula.first

                        if question:
                            if self.prove(formula):
                                print('Yes')
                            elif self.prove(Expression.fromstring(f'not {formula}')):
                                print('No')
                            else:
                                print('I don\'t know')
                        elif assertion:
                            if self.prove(formula):
                                print('I already know that')
                            elif self.prove(Expression.fromstring(f'not {formula}')):
                                print('I don\'t buy that')
                            else:
                                self.KB.append(formula)
                                print('I learned something')
                        else:
                            print('I don\'t know what to do with that')
                    except Exception as e:
                        continue

        except StopIteration:
            pass
if __name__ == '__main__':
    agent = Agent()
    agent.dialog()                         


>>>  Alice is a student
I learned something
>>>  Is Alice a student ?
Yes
>>>  Is Bob a student ?
I don't know
>>>  Students are people
I learned something
>>>  Alice is a person
I already know that
>>>  Alice is not a person
I don't buy that
>>>  Alice is from Phoenix
I learned something
>>>  Is Alice from a city ?
I don't know
>>>  Phoenix is a hot city
I already know that
>>>  Is Alice from a city ?
I don't know
>>>  Cities are places
I learned something
>>>  If it is snowing then it is cold
>>>  It is not cold
>>>  Is it snowing ?
I don't know
>>>  If a person is from a hot place and it is cold then she is not happy
>>>  Is it snowing ?
I don't know
>>>  Alice is happy
I learned something
>>>  Is Alice happy ?
>>>  Is it snowing ?
I don't know


### Partie 2

Augmenter la grammaire pour obtenir des réponses autres que 'yes/no' à vos questions.

Par exemple :
- who is a student ?
- what places are hot ?

In [48]:
sents2 = [
    'Alice is a student',    
    'Bob is a student',   
    'Cyril is a student',    
    'who is a student ?',   
    'Phoenix is a hot place',      
    'Meknes is a hot place',  
    'Marrakech is a hot place',   
    'what places are hot ?'       
    ]

In [63]:

class Agent:

    def __init__(self, grammar='grammar/dialog.fcfg', source=iter(SentencesIterator(sents2))):
        self.KB = []
        with open(grammar, 'r') as file:
            data = file.read()
        self.grammar = nltk.grammar.FeatureGrammar.fromstring(data)
        self.parser = nltk.FeatureChartParser(self.grammar, trace=0)
        self.source = source

    def prove(self, formula):
        tp = Prover9Command(formula, self.KB)
        # ATTENTION: ajuster le chemin à l'exécutable
        tp._prover.config_prover9('c:/Program Files (x86)/Prover9-Mace4/bin-win32/prover9.exe')
        return tp.prove()
    
    def handle_wh_question(self, formula):
        results = []
        predicate = str(formula).split('(')[0]  # Extrait le prédicat de la formule
        print(f"Recherche de prédicat dans la KB: {predicate}")

        for fact in self.KB:
            print(f"Vérification du fait dans la KB: {fact}")
            if predicate in str(fact):
                entity = str(fact).split('(')[1].split(')')[0]
                results.append(entity)

        print(f"Résultats trouvés: {results}")
        return results

    def dialog(self):
        try:
                while True:
                    sentence = next(self.source)
                    try:
                        parsed_tree = next(self.parser.parse(sentence.split()))
                        formula = parsed_tree.label()['SEM']
                        assert isinstance(formula, AndExpression)
                        question = (formula.second == ConstantExpression(Variable('question')))
                        assertion = (formula.second == ConstantExpression(Variable('assertion')))
                        wh_question = 'SELECT' in str(formula.first)
                        formula = formula.first
                        

                        if question:
                            if wh_question:
                                print(f"Traitement d'une question WH: {sentence}")
                                answers = self.handle_wh_question(formula)
                                for answer in answers:
                                    print(answer)
                            else :
                                if self.prove(formula):
                                    print('Yes')
                                elif self.prove(Expression.fromstring(f'not {formula}')):
                                    print('No')
                                else:
                                    print('I don\'t know')
                        elif assertion:
                            if self.prove(formula):
                                print('I already know that')
                            elif self.prove(Expression.fromstring(f'not {formula}')):
                                print('I don\'t buy that')
                            else:
                                self.KB.append(formula)
                                print('I learned something')
                        else:
                            print('I don\'t know what to do with that')
                    except Exception as e:
                        continue

        except StopIteration:
            pass
if __name__ == '__main__':
    agent = Agent()
    agent.dialog()                         


>>>  Alice is a student
I learned something
>>>  Bob is a student
I learned something
>>>  Cyril is a student
I learned something
>>>  who is a student ?
>>>  Phoenix is a hot place
I learned something
>>>  Meknes is a hot place
I learned something
>>>  Marrakech is a hot place
I already know that
>>>  what places are hot ?
